In [1]:
import torchvision

In [2]:
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 113MB/s]


In [3]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
import torch
from PIL import Image
from torchvision import transforms

import requests
from io import BytesIO

# 전처리 정의
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# 이미지 URL
url = "https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQ0njRy35t-uubPwRniphCvVI-b4-Nxj0zntBIO7gmENTa39iMV"

# 이미지 다운로드
response = requests.get(url)
image = Image.open(BytesIO(response.content))

# 이미지 로드 및 전처리
# image = Image.open("example.jpg")

# RGBA 이미지를 RGB로 변환
if image.mode == 'RGBA':
    image = image.convert('RGB')

input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # 배치 차원 추가

# 추론 모드로 전환
model.eval()

# 예측
with torch.no_grad():
    output = model(input_batch)

# 결과 확인
_, predicted = torch.max(output.data, 1)
print(_, predicted)

tensor([20.4759]) tensor([5])


In [5]:
# ImageNet 클래스 레이블 불러오기
import requests

# ImageNet 클래스 목록 URL
url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"

# 클래스 레이블 다운로드
response = requests.get(url)
labels = response.text.split('\n')

# 916번 클래스 확인
predicted_class = labels[5]
print(f"클래스 인덱스 5: {predicted_class}")

클래스 인덱스 5: electric ray


#2 - Hugging Face

In [6]:
!pip install transformers

In [7]:
!pip install pillow

In [8]:
from transformers import AutoImageProcessor, ResNetForImageClassification
from PIL import Image

# 모델과 프로세서 불러오기
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-18")

# 이미지 URL
url = "https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQ0njRy35t-uubPwRniphCvVI-b4-Nxj0zntBIO7gmENTa39iMV"

# 이미지 다운로드
response = requests.get(url)
image = Image.open(BytesIO(response.content))

# 이미지 처리 및 예측
# image = Image.open("https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQ0njRy35t-uubPwRniphCvVI-b4-Nxj0zntBIO7gmENTa39iMV")
inputs = processor(image, return_tensors="pt")

outputs = model(**inputs)
logits = outputs.logits

# 예측된 클래스 확인
predicted_class = logits.argmax(-1).item()
print(predicted_class)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

5


##검증

In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
from datasets import load_dataset

# # imagenet-1k는 HuggingFace에서 제공하는 ImageNet의 subset입니다
# dataset = load_dataset("imagenet-1k", split="validation")


from torchvision.datasets import CIFAR100

# CIFAR-100 (100개 클래스)
dataset = CIFAR100(root='./data', train=False, download=True)

100%|██████████| 169M/169M [00:08<00:00, 20.0MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


In [12]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

def evaluate_on_cifar100(model, dataset, batch_size=32, num_epochs=5):
    val_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    best_top1_acc = 0
    best_top5_acc = 0

    for epoch in range(num_epochs):
        correct_top1 = 0
        correct_top5 = 0
        total = 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)

                # Top-1 accuracy
                _, predicted = torch.max(outputs.data, 1)
                correct_top1 += (predicted == labels).sum().item()

                # Top-5 accuracy
                _, top5_predicted = outputs.topk(5, 1, largest=True, sorted=True)
                labels = labels.view(labels.size(0), -1).expand_as(top5_predicted)
                correct_top5 += (top5_predicted == labels).sum().float().item()

                total += labels.size(0)

        # Calculate accuracies
        top1_accuracy = 100 * correct_top1 / total
        top5_accuracy = 100 * correct_top5 / (total * 5)  # 각 이미지당 5개의 예측을 고려

        print(f'Epoch {epoch+1}:')
        print(f'Top-1 Accuracy: {top1_accuracy:.2f}%')
        print(f'Top-5 Accuracy: {top5_accuracy:.2f}%')
        print('-' * 50)

        # Update best accuracies
        if top1_accuracy > best_top1_acc:
            best_top1_acc = top1_accuracy
        if top5_accuracy > best_top5_acc:
            best_top5_acc = top5_accuracy

    print('\nBest Results:')
    print(f'Best Top-1 Accuracy: {best_top1_acc:.2f}%')
    print(f'Best Top-5 Accuracy: {best_top5_acc:.2f}%')

    return best_top1_acc, best_top5_acc

# 데이터 전처리 정의
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761]
    )
])

# 데이터셋 로드
dataset = torchvision.datasets.CIFAR100(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# 모델 설정
model = torchvision.models.resnet18(pretrained=True)
model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = torch.nn.Identity()
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 100)

# 평가 실행 (5 에포크로 설정)
top1_acc, top5_acc = evaluate_on_cifar100(model, dataset, num_epochs=5)

Files already downloaded and verified


Epoch 1/5: 100%|██████████| 313/313 [00:08<00:00, 37.59it/s]


Epoch 1:
Top-1 Accuracy: 1.09%
Top-5 Accuracy: 1.06%
--------------------------------------------------


Epoch 2/5: 100%|██████████| 313/313 [00:04<00:00, 70.18it/s]


Epoch 2:
Top-1 Accuracy: 1.09%
Top-5 Accuracy: 1.06%
--------------------------------------------------


...

Epoch 3:
Top-1 Accuracy: 1.09%
Top-5 Accuracy: 1.06%
--------------------------------------------------


Epoch 4/5: 100%|██████████| 313/313 [00:04<00:00, 72.16it/s]


Epoch 4:
Top-1 Accuracy: 1.09%
Top-5 Accuracy: 1.06%
--------------------------------------------------


Epoch 5/5: 100%|██████████| 313/313 [00:04<00:00, 75.33it/s]

Epoch 5:
Top-1 Accuracy: 1.09%
Top-5 Accuracy: 1.06%
--------------------------------------------------

Best Results:
Best Top-1 Accuracy: 1.09%
Best Top-5 Accuracy: 1.06%


In [13]:
import torch.optim as optim
import torch.nn as nn

def train_model(model, train_loader, val_loader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    for epoch in range(epochs):
        # 학습 모드
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # # 평가 모드
        # model.eval()

        # 여기에 이전의 evaluate_on_cifar100 함수 내용 추가

def evaluate_on_cifar100(model, dataset, batch_size=32, num_epochs=5):
    val_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    best_top1_acc = 0
    best_top5_acc = 0

    for epoch in range(num_epochs):
        correct_top1 = 0
        correct_top5 = 0
        total = 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)

                # Top-1 accuracy
                _, predicted = torch.max(outputs.data, 1)
                correct_top1 += (predicted == labels).sum().item()

                # Top-5 accuracy
                _, top5_predicted = outputs.topk(5, 1, largest=True, sorted=True)
                labels = labels.view(labels.size(0), -1).expand_as(top5_predicted)
                correct_top5 += (top5_predicted == labels).sum().float().item()

                total += labels.size(0)

        # Calculate accuracies
        top1_accuracy = 100 * correct_top1 / total
        top5_accuracy = 100 * correct_top5 / (total * 5)  # 각 이미지당 5개의 예측을 고려

        print(f'Epoch {epoch+1}:')
        print(f'Top-1 Accuracy: {top1_accuracy:.2f}%')
        print(f'Top-5 Accuracy: {top5_accuracy:.2f}%')
        print('-' * 50)

        # Update best accuracies
        if top1_accuracy > best_top1_acc:
            best_top1_acc = top1_accuracy
        if top5_accuracy > best_top5_acc:
            best_top5_acc = top5_accuracy

    print('\nBest Results:')
    print(f'Best Top-1 Accuracy: {best_top1_acc:.2f}%')
    print(f'Best Top-5 Accuracy: {best_top5_acc:.2f}%')

    return best_top1_acc, best_top5_acc

# 데이터 전처리 정의
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5071, 0.4867, 0.4408],
        std=[0.2675, 0.2565, 0.2761]
    )
])

# 데이터셋 로드
dataset = torchvision.datasets.CIFAR100(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# 모델 설정
model = torchvision.models.resnet18(pretrained=True)
model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = torch.nn.Identity()
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 100)

# 평가 실행 (5 에포크로 설정)
top1_acc, top5_acc = evaluate_on_cifar100(model, dataset, num_epochs=5)

Files already downloaded and verified


Epoch 1/5: 100%|██████████| 313/313 [00:06<00:00, 48.22it/s]


Epoch 1:
Top-1 Accuracy: 1.05%
Top-5 Accuracy: 0.99%
--------------------------------------------------


Epoch 2/5: 100%|██████████| 313/313 [00:05<00:00, 54.54it/s]


Epoch 2:
Top-1 Accuracy: 1.05%
Top-5 Accuracy: 0.99%
--------------------------------------------------


...

Epoch 3:
Top-1 Accuracy: 1.05%
Top-5 Accuracy: 0.99%
--------------------------------------------------


Epoch 4/5: 100%|██████████| 313/313 [00:04<00:00, 75.02it/s]


Epoch 4:
Top-1 Accuracy: 1.05%
Top-5 Accuracy: 0.99%
--------------------------------------------------


Epoch 5/5: 100%|██████████| 313/313 [00:06<00:00, 48.18it/s]

Epoch 5:
Top-1 Accuracy: 1.05%
Top-5 Accuracy: 0.99%
--------------------------------------------------

Best Results:
Best Top-1 Accuracy: 1.05%
Best Top-5 Accuracy: 0.99%


In [14]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# 데이터 전처리 정의
transform = transforms.Compose([
    transforms.Resize(224),  # ResNet의 기본 입력 크기
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet 통계값 사용
        std=[0.229, 0.224, 0.225]
    )
])

# 데이터셋 로드
dataset = torchvision.datasets.CIFAR100(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

def evaluate_on_cifar100(model, dataset, batch_size=32, num_epochs=5):
    val_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    best_top1_acc = 0
    best_top5_acc = 0

    for epoch in range(num_epochs):
        correct_top1 = 0
        correct_top5 = 0
        total = 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)

                # Top-1 정확도
                _, predicted = torch.max(outputs.data, 1)
                correct_top1 += (predicted == labels).sum().item()

                # Top-5 정확도
                _, top5_pred = outputs.topk(5, 1, largest=True, sorted=True)
                for i in range(labels.size(0)):
                    if labels[i] in top5_pred[i]:
                        correct_top5 += 1

                total += labels.size(0)

        epoch_top1_acc = 100 * correct_top1 / total
        epoch_top5_acc = 100 * correct_top5 / total

        print(f'Epoch {epoch + 1}:')
        print(f'Top-1 Accuracy: {epoch_top1_acc:.2f}%')
        print(f'Top-5 Accuracy: {epoch_top5_acc:.2f}%')

        best_top1_acc = max(best_top1_acc, epoch_top1_acc)
        best_top5_acc = max(best_top5_acc, epoch_top5_acc)

    print('\nBest Results:')
    print(f'Best Top-1 Accuracy: {best_top1_acc:.2f}%')
    print(f'Best Top-5 Accuracy: {best_top5_acc:.2f}%')

    return best_top1_acc, best_top5_acc

# 모델 로드
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

# 마지막 층만 수정
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 100)

# 평가 실행
accuracy = evaluate_on_cifar100(model, dataset)

Files already downloaded and verified


100%|██████████| 313/313 [00:37<00:00,  8.40it/s]


Epoch 1:
Top-1 Accuracy: 0.82%
Top-5 Accuracy: 4.44%


100%|██████████| 313/313 [00:23<00:00, 13.11it/s]


Epoch 2:
Top-1 Accuracy: 0.82%
Top-5 Accuracy: 4.44%


...

Epoch 3:
Top-1 Accuracy: 0.82%
Top-5 Accuracy: 4.44%


100%|██████████| 313/313 [00:21<00:00, 14.67it/s]


Epoch 4:
Top-1 Accuracy: 0.82%
Top-5 Accuracy: 4.44%


100%|██████████| 313/313 [00:23<00:00, 13.20it/s]

Epoch 5:
Top-1 Accuracy: 0.82%
Top-5 Accuracy: 4.44%

Best Results:
Best Top-1 Accuracy: 0.82%
Best Top-5 Accuracy: 4.44%
